# Análisis y visualización de la base de datos de "Dialectos del español"

![](https://www.dialectosdelespanol.org/android-icon-144x144.png)

Este análisis ha sido realizado el 31 de marzo del 2022 por <b><a href="www.h4ck1ng.science">Carlos Vivar Ríos (UNIL, h4ck1ng.science)</a></b> en colaboración con <b>Cristina Pena Rueda (Section d'espagnol, UNIL).</b>

El objetivo de esta página web es proporcionar una forma rápida y segura de poder analizar los resultados de la base de datos. Es necesario contar con los archivos`.pickle`, `.csv` y `.geojson` para poder ejecutar este código. Los archivos no serán subidos a ningún servidor y todo el análisis se realizará en tu navegador gracias a la tecnología WebAssembly. Todo dato se borrará al cerra la pestaña. Debido al ser una tecnología en desarrollo se recomienda el uso de Google Chrome o Mozilla Firefox. 

<b>Cuaderno en proceso de construcción</b>

## Instalación de las dependencias

In [ ]:
import piplite
await piplite.install(['plotly', 'ipywidgets','openpyxl', 'itables'])

In [ ]:
import ipywidgets as widgets
import io
import pandas as pd
import plotly.express as px
import io
from IPython.core.display import display, HTML
#import plotly.graph_objects as go
from itables import init_notebook_mode
from itables import show as ishow
init_notebook_mode(all_interactive=True)

In [ ]:
display(HTML("<style>.jp-Notebook { padding-left: 100px !important; padding-right: 100px !important; }</style>"))

# Carga de los datos

### Carga del base de datos en formato excel/Pickle

Es necesario esperar a que el archivo este cargado para continuar. `Upload (0)` pasara a `Upload (1)`.

In [ ]:
uploader = widgets.FileUpload(
    accept='', 
    multiple=False  
)
display(uploader)

In [ ]:
input_file = list(uploader.value.values())[0]
content = input_file['content']
content = io.BytesIO(content)
df = pd.read_pickle(content).iloc[:200000]
df.shape
#ishow(df.iloc[:1000])

In [ ]:
# ## This just in case the file is a csv
# input_file = list(uploader.value.values())[0]
# content = input_file['content']
# content = io.StringIO(content.decode('utf-8'))
# df = pd.read_csv(content, names=['lat', 'lon', 'country_code', 'city'], sep=';')
# df.shape

## Visualización de la tabla

In [ ]:
subdf = df.iloc[:1000]

In [ ]:
#ishow(subdf)

## Visualización del mapa

Selección de un campo de coordenadas por ejemplo "Vivo en". Para poder realizar esta visualización debes proporcionar tu clave API de Mapbox. Se puede conseguir gratuitamente: [MapBox](https://www.mapbox.com/)

In [ ]:
ACCESS_TOKEN = widgets.Text(
    value='Hello World',
    placeholder='Type your token',
    description='String:',
    disabled=False
)
display(ACCESS_TOKEN)

In [ ]:
px.set_mapbox_access_token(ACCESS_TOKEN.value)

Selección de columnas y de países de interés

In [ ]:
countries_of_interest = ['Spain', 'Cuba', 'Puerto Rico', 'Dominican Republic', 'Paraguay']
dfmap =  df[df['Vivo en _predicted_country'].isin(countries_of_interest)]
dfmap = dfmap[['Juan habla mal...', 'Vivo en _lat', 'Vivo en _lon', 'Vivo en _predicted_country']].dropna() #'Vivo en _predicted_city' 

### Visualizaci'on por pa'is

In [ ]:
fig = px.scatter_mapbox(dfmap, lat="Vivo en _lat", lon="Vivo en _lon", color='Juan habla mal...', hover_name='Vivo en _predicted_country',
                        hover_data=['Vivo en _predicted_country'], #, 'Vivo en _predicted_city'
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=2, zoom=1, opacity=1, width=1500, height=800)

#fig.update_layout(coloraxis_showscale=False)
#fig.update_layout(showlegend=True)
#fig.show()

### Category Vis

In [ ]:
fig = px.scatter_mapbox(dfmap, lat="Vivo en _lat", lon="Vivo en _lon", color='Juan habla mal...', hover_name='Vivo en _predicted_country',
                        hover_data=['Juan habla mal...'], #, 'Vivo en _predicted_city'
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=2, zoom=1, opacity=1, width=1500, height=800)

#fig.update_layout(coloraxis_showscale=False)
fig.update_layout(showlegend=True)
#fig.show()

### Densidad de variante

In [ ]:
dfmap['Juan habla mal...'].unique()

In [ ]:
dfmap['heat'] = 1
subdf = dfmap[dfmap['Juan habla mal...'] == 'de mí.']

In [ ]:
fig = px.density_mapbox(subdf, lat="Vivo en _lat", lon="Vivo en _lon", z='heat', radius=1,
                        center=dict(lat=0, lon=180),
                        zoom=1,width=1500, height=800,opacity=0.5)#,
                        #mapbox_style="stamen-terrain")
fig.show()